## Visualizing the Map and Clusters of the Places

In [1]:
# importing old libraries (also used in the two first parts)
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs

In [2]:
# importing new libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Load the pandas dataframe (created in part 2 of the assignment)

In [7]:
df_toronto=pd.read_csv("toronto_base.csv")
df_toronto.head(7)

,Unnamed: 0,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,0,Scarborough,Woburn,M1G,43.770992,-79.216917
1,1,North York,Glencairn,M6B,43.709577,-79.445073
2,2,Central Toronto,"Deer Park, Rathnelly, South Hill",M4V,43.686412,-79.400049
3,3,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou...",M5V,43.628947,-79.394420
4,4,Downtown Toronto,"Toronto Islands, Union Station",M5J,43.640816,-79.381752
5,5,Downtown Toronto,Ryerson,M5B,43.657162,-79.378937
6,6,North York,Bedford Park,M5M,43.733283,-79.419750


### Create a map of Toronto

In [8]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

### Create a new data frame with neighborhoods in Scarborough 

In [9]:
# @hiddel_cell
CLIENT_ID = '3UQAO5E1WY5ITJFJLWPNBOXXKSB13RD5NRJ1CN04YWB4HM3F' # your Foursquare ID
CLIENT_SECRET = 'JUEG0PHBMY3HII1IJR4NPBZRKZVD30PNAQMPVSVXCHOJ50JE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



In [10]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Unnamed: 0,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,0,Scarborough,Woburn,M1G,43.770992,-79.216917
1,11,Scarborough,Agincourt,M1S,43.794200,-79.262029
2,19,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
3,23,Scarborough,"Morningside, West Hill",M1E,43.763573,-79.188711
4,24,Scarborough,Birch Cliff,M1N,43.692657,-79.264848
5,27,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
6,29,Scarborough,"Cliffcrest, Cliffside",M1M,43.716316,-79.239476


### Create a map of Scarborough and its neighbourhoods

In [11]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [12]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

### k-means to cluster the neighborhoods 

In [30]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(16)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([0, 2, 0, 1, 1, 1, 1, 3, 1, 1])

### Include kmeans.labels_ into the  Scarborough dataframe

In [31]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,Unnamed: 0,Borough,Neighbourhood,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Scarborough,Woburn,M1G,43.770992,-79.216917,0,Coffee Shop,Soccer Field,Korean Restaurant,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
1,11,Scarborough,Agincourt,M1S,43.794200,-79.262029,2,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,19,Scarborough,Scarborough Village,M1J,43.744734,-79.239476,0,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,23,Scarborough,"Morningside, West Hill",M1E,43.763573,-79.188711,1,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint
4,24,Scarborough,Birch Cliff,M1N,43.692657,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
5,27,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Convenience Store,Grocery Store
6,29,Scarborough,"Cliffcrest, Cliffside",M1M,43.716316,-79.239476,1,American Restaurant,Motel,Indian Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
7,36,Scarborough,"Agincourt North, Milliken",M1V,43.815252,-79.284577,3,Playground,Park,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
8,41,Scarborough,"Maryvale, Wexford",M1R,43.750071,-79.295849,1,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
9,49,Scarborough,Steeles West,M1W,43.799525,-79.318389,1,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Coffee Shop,Japanese Restaurant,Sandwich Place,Breakfast Spot,Electronics Store,Pizza Place,Pharmacy


### Visualize the clusters in the map

In [32]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each of the five clusters

In [33]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.216917,0,Coffee Shop,Soccer Field,Korean Restaurant,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,Scarborough,-79.239476,0,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [34]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,-79.188711,1,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint
4,Scarborough,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
5,Scarborough,-79.239476,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Convenience Store,Grocery Store
6,Scarborough,-79.239476,1,American Restaurant,Motel,Indian Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
8,Scarborough,-79.295849,1,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
9,Scarborough,-79.318389,1,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Coffee Shop,Japanese Restaurant,Sandwich Place,Breakfast Spot,Electronics Store,Pizza Place,Pharmacy
10,Scarborough,-79.273304,1,Indian Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Bar,Breakfast Spot,Hakka Restaurant,Grocery Store,General Entertainment
13,Scarborough,-79.160497,1,Moving Target,Bar,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
14,Scarborough,-79.205636,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,Scarborough,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


In [35]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]    

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,-79.262029,2,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
12,Scarborough,-79.304302,2,Pizza Place,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Fast Food Restaurant,Pharmacy,Noodle House,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop


In [36]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,-79.284577,3,Playground,Park,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [37]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,-79.284577,4,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,College Stadium,Grocery Store,General Entertainment
